# Intsall Package

In [2]:
# Install relevant packages, if not yet
!pip3 install awswrangler
!pip3 install pandasql

  Using cached awswrangler-2.16.1-py3-none-any.whl (248 kB)
  Using cached pandas-1.3.5-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.3 MB)
  Using cached opensearch_py-1.1.0-py2.py3-none-any.whl (207 kB)
  Using cached gremlinpython-3.6.0-py2.py3-none-any.whl (72 kB)
  Using cached pyarrow-7.0.0-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (26.7 MB)
  Using cached PyMySQL-1.0.2-py3-none-any.whl (43 kB)
  Using cached jsonpath_ng-1.5.3-py3-none-any.whl (29 kB)
  Using cached pg8000-1.29.1-py3-none-any.whl (34 kB)
  Using cached backoff-2.1.2-py3-none-any.whl (14 kB)
  Using cached redshift_connector-2.0.908-py3-none-any.whl (112 kB)
  Using cached requests_aws4auth-1.1.2-py2.py3-none-any.whl (24 kB)
  Using cached progressbar2-4.0.0-py2.py3-none-any.whl (26 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Using cached aenum-3.1.11-py3-none-any.whl (131 kB)
  Using cached scramp-1.4.1-py3-none-any.whl (8.5 kB)
  Using cached python_utils-3.3.3

In [3]:
from data_connector import *
import numpy as np
import pandas as pd

# Import Packages and Connect to RedShift

In [3]:
# Fill in the relevant credentials for Redshift cluster(s) and S3 bucket(s) to use
role_arn = 'arn:aws:iam::848271137894:role/RedShiftAccessFromSageMakerAccount'
role_session_name = 'AssumeRoleSession1'
secret_name = 'arn:aws:secretsmanager:us-west-2:848271137894:secret:Redshift-DLL-Sagemaker-Secret-nHWV37'
dbname = 'ucicanvas'
region_name = 'us-west-2'
bucket_name = 'uci-dll-mellon-redshift-unload-us-west-2'
unload_iam_role = 'arn:aws:iam::848271137894:role/uci-write-to-dll-mellon-redshift-unload-s3-bucket-role,arn:aws:iam::686457044982:role/uci-oit-dwh-redshift-to-mellon-s3-unload-role'

In [4]:
# Initiate a connector object
redshift_s3_connect = RedshiftS3Connector(role_arn, role_session_name, secret_name, dbname, bucket_name, region_name, unload_iam_role)

# Get the discussion_post layer2 table from Fall 2020 to Spring 2022

In [14]:
get_df_query = f'''
WITH roster AS
         (SELECT DISTINCT mellon_id,
                          sis_term_code,
                          instruc_begins::DATE,
                          (instruc_ends::DATE + 9) AS final_ends,
                          canvas_course_id,
                          sis_course_code,
                          canvas_course_section_id,
                          sis_course_section_code
          FROM mellon_layer2.enrollment er
                   LEFT JOIN mellon_layer2.term_dates td ON er.sis_term_code = td.term_desc
          WHERE is_valid_user
            AND is_valid_course_section
            AND enrollment_type = \\'StudentEnrollment\\')
SELECT dp.*, rs.sis_term_code AS term 
FROM mellon_layer2.v_discussion_post AS dp INNER JOIN 
(SELECT DISTINCT mellon_id, canvas_course_id, instruc_begins, final_ends, sis_term_code FROM roster) AS rs
                              ON dp.canvas_course_id = rs.canvas_course_id AND dp.mellon_id = rs.mellon_id
WHERE CONVERT_TIMEZONE(\\'PST\\', dp.discussion_post_created_at) BETWEEN \\'2020-10-01 00:00:00\\' AND \\'2022-06-10 00:00:00\\' 
        AND CONVERT_TIMEZONE(\\'PST\\', dp.discussion_post_created_at) BETWEEN rs.instruc_begins AND rs.final_ends
        AND dp.discussion_post_deleted_at IS NULL;'''
print(get_df_query)


WITH roster AS
         (SELECT DISTINCT mellon_id,
                          sis_term_code,
                          instruc_begins::DATE,
                          (instruc_ends::DATE + 9) AS final_ends,
                          canvas_course_id,
                          sis_course_code,
                          canvas_course_section_id,
                          sis_course_section_code
          FROM mellon_layer2.enrollment er
                   LEFT JOIN mellon_layer2.term_dates td ON er.sis_term_code = td.term_desc
          WHERE is_valid_user
            AND is_valid_course_section
            AND enrollment_type = \'StudentEnrollment\')
SELECT dp.*, rs.sis_term_code AS term 
FROM mellon_layer2.v_discussion_post AS dp INNER JOIN 
(SELECT DISTINCT mellon_id, canvas_course_id, instruc_begins, final_ends, sis_term_code FROM roster) AS rs
                              ON dp.canvas_course_id = rs.canvas_course_id AND dp.mellon_id = rs.mellon_id
WHERE CONVERT_TIMEZONE(\'PST\', d

In [7]:
file_in_s3 = 'discussion_post_2020_2022.csv'

In [15]:
redshift_s3_connect.unload_from_redshift_to_s3(get_df_query, file_in_s3, 'csv')

Waiter DataAPIExecution failed: Max attempts exceeded
Status: STARTED. Excution time: 0 miliseconds
{'ClusterIdentifier': 'uci-datalake-canvas-cluster-v1-redshiftcluster-a9h29wndif8j', 'CreatedAt': datetime.datetime(2022, 7, 18, 22, 2, 43, 172000, tzinfo=tzlocal()), 'Duration': -1, 'HasResultSet': False, 'Id': '04b8f006-7a0f-45ec-993a-b51c16364386', 'QueryString': "UNLOAD ('\nWITH roster AS\n         (SELECT DISTINCT mellon_id,\n                          sis_term_code,\n                          instruc_begins::DATE,\n                          (instruc_ends::DATE + 9) AS final_ends,\n                          canvas_course_id,\n                          sis_course_code,\n                          canvas_course_section_id,\n                          sis_course_section_code\n          FROM mellon_layer2.enrollment er\n                   LEFT JOIN mellon_layer2.term_dates td ON er.sis_term_code = td.term_desc\n          WHERE is_valid_user\n            AND is_valid_course_section\n       

In [8]:
file_names = [file_in_s3]
file_type = 'csv'
discussion_post = redshift_s3_connect.read_from_s3(file_names, file_type)
discussion_post.head(5)

,discussion_post_id,mellon_id,canvas_course_id,discussion_post_content,discussion_post_created_at,discussion_post_updated_at,discussion_post_deleted_at,discussion_post_depth,parent_discussion_post_id,discussion_topic_id,discussion_topic_title,is_announcement,discussion_topic_created_at,discussion_topic_posted_at,discussion_topic_delayed_post_at,discussion_topic_pinned,user_group_id,assignment_id,term
0,3414032,179827,34185,<p>If we need any other form of communication ...,2021-01-08 19:43:58.995,2021-01-08 19:43:58.995,NaN,1,NaN,480258,Scheduling a Meeting,NaN,2021-01-08 19:26:01.007,2021-01-08 19:26:01.008,NaN,f,113255.0,NaN,Winter 2021
1,3429509,179827,34185,<p>Sure thing. Here it is. This is the new lin...,2021-01-11 04:33:37.551,2021-01-11 19:12:19.077,NaN,2,3414032.0,480258,Scheduling a Meeting,NaN,2021-01-08 19:26:01.007,2021-01-08 19:26:01.008,NaN,f,113255.0,NaN,Winter 2021
2,3884633,183405,36395,"<p>Hello Everyone,&nbsp;</p>\n<p>When I first ...",2021-04-04 05:45:42.943,2021-04-04 05:45:42.943,NaN,1,NaN,534024,✔ Welcome Discussion Board,NaN,2021-03-28 22:45:51.515,NaN,NaN,t,NaN,735497.0,Spring 2021
3,5199137,199149,44320,<p>Hi everyone! I'm fine with any of those met...,2022-03-30 18:38:31.989,2022-03-30 18:38:31.989,NaN,1,NaN,726612,communication,NaN,2022-03-29 23:00:05.684,2022-03-29 23:00:05.685,NaN,f,158705.0,NaN,Spring 2022
4,3304351,173227,29368,lillya@uci.edu ty!,2020-12-04 00:54:26.181,2020-12-04 00:54:26.181,NaN,1,NaN,451878,Paper presentation,NaN,2020-12-03 23:27:25.27,2020-12-03 23:27:25.27,NaN,f,97862.0,NaN,Fall 2020


In [9]:
discussion_post.to_csv('discussion_post_2020_2022.csv')

In [10]:
df = pd.read_csv('discussion_post_2020_2022.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'discussion_post_2020_2022.csv'

In [8]:
print(df['discussion_post_content'][0])

<p>Hello Chengzhi, I highly agree with your perspective on how conceptual analysis can be fruitful. I especially enjoyed your example between humans and primates because our studies of primates we have grown a better understanding of or primal parts of human behavior. And as you stated by using our closest genetic relative we've been able to see some aspects of our society in their hierarchies.</p>


In [11]:
df_small = discussion_post.sample(n=100000)

In [12]:
df_small.to_csv('discussion_post_2020_2022_small.csv')

In [22]:
discussion_post.groupby(by='term')['discussion_post_id'].count().reset_index(name='discussion_post_cnt').reindex([0, 4, 2, 1, 5, 3])

,term,discussion_post_cnt
0,Fall 2020,300551
4,Winter 2021,268359
2,Spring 2021,250647
1,Fall 2021,140963
5,Winter 2022,116430
3,Spring 2022,91394


In [11]:
temp_quary = f'''
WITH roster AS
         (SELECT DISTINCT mellon_id,
                          sis_term_code,
                          instruc_begins::DATE,
                          (instruc_ends::DATE + 9) AS final_ends,
                          canvas_course_id,
                          sis_course_code,
                          canvas_course_section_id,
                          sis_course_section_code
          FROM mellon_layer2.enrollment er
                   LEFT JOIN mellon_layer2.term_dates td ON er.sis_term_code = td.term_desc
          WHERE is_valid_user
            AND is_valid_course_section
            AND enrollment_type = \\'StudentEnrollment\\')
SELECT dp.*, rs.sis_term_code AS term 
FROM mellon_layer2.v_discussion_post AS dp INNER JOIN 
(SELECT DISTINCT mellon_id, canvas_course_id, instruc_begins, final_ends, sis_term_code FROM roster) AS rs
                              ON dp.canvas_course_id = rs.canvas_course_id AND dp.mellon_id = rs.mellon_id
WHERE CONVERT_TIMEZONE(\\'PST\\', dp.discussion_post_created_at) BETWEEN \\'2018-09-24 00:00:00\\' AND \\'2020-06-12 00:00:00\\' 
        AND CONVERT_TIMEZONE(\\'PST\\', dp.discussion_post_created_at) BETWEEN rs.instruc_begins AND rs.final_ends
        AND dp.discussion_post_deleted_at IS NULL;'''

In [5]:
file_in_s3 = 'discussion_post_2018_2020.csv'

In [13]:
redshift_s3_connect.unload_from_redshift_to_s3(temp_quary, file_in_s3, 'csv')

Done waiting to finish Data API.
Status: FINISHED. Excution time: 26821 miliseconds
{'ClusterIdentifier': 'uci-datalake-canvas-cluster-v1-redshiftcluster-a9h29wndif8j', 'CreatedAt': datetime.datetime(2022, 7, 18, 23, 54, 21, 804000, tzinfo=tzlocal()), 'Duration': 26821495956, 'HasResultSet': False, 'Id': 'b5c83614-86af-43e1-8b2a-adcd64aa8e4e', 'QueryString': "UNLOAD ('\nWITH roster AS\n         (SELECT DISTINCT mellon_id,\n                          sis_term_code,\n                          instruc_begins::DATE,\n                          (instruc_ends::DATE + 9) AS final_ends,\n                          canvas_course_id,\n                          sis_course_code,\n                          canvas_course_section_id,\n                          sis_course_section_code\n          FROM mellon_layer2.enrollment er\n                   LEFT JOIN mellon_layer2.term_dates td ON er.sis_term_code = td.term_desc\n          WHERE is_valid_user\n            AND is_valid_course_section\n            A

In [6]:
file_names = [file_in_s3]
file_type = 'csv'
discussion_post_2018_2020 = redshift_s3_connect.read_from_s3(file_names, file_type)
discussion_post_2018_2020.head(5)

,discussion_post_id,mellon_id,canvas_course_id,discussion_post_content,discussion_post_created_at,discussion_post_updated_at,discussion_post_deleted_at,discussion_post_depth,parent_discussion_post_id,discussion_topic_id,discussion_topic_title,is_announcement,discussion_topic_created_at,discussion_topic_posted_at,discussion_topic_delayed_post_at,discussion_topic_pinned,user_group_id,assignment_id,term
0,1419944,192222,15234,<p>After watching the silent film UTC directed...,2019-01-25 07:31:30.422,2019-01-25 07:31:30.422,NaN,1,NaN,177413,Week Three Discussion Thread,NaN,2019-01-15 17:30:20.143,2019-01-15 17:46:07.122,NaN,f,NaN,NaN,Winter 2019
1,1486847,192222,15234,"<p>The ""tragedy"" that happened in Tenssee Will...",2019-02-22 05:01:03.103,2019-02-22 05:01:03.103,NaN,1,NaN,177419,Week Seven Discussion Thread,NaN,2019-01-15 17:40:25.37,2019-02-15 20:59:27.806,NaN,f,NaN,NaN,Winter 2019
2,1438459,192222,15234,<p>If you want to find a comedy master who can...,2019-02-01 06:10:56.836,2019-02-01 06:10:56.836,NaN,1,NaN,177415,Week Four Discussion Thread,NaN,2019-01-15 17:32:12.246,2019-01-25 16:08:56.636,NaN,f,NaN,NaN,Winter 2019
3,1528679,192222,15234,"In Julia Jarcho's dreamland, young Haley and h...",2019-03-15 05:44:01.065,2019-03-15 05:44:01.065,NaN,1,NaN,177425,Week Ten Discussion Thread,NaN,2019-01-15 17:45:51.312,2019-03-08 19:10:54.891,NaN,f,NaN,NaN,Winter 2019
4,1502377,192222,15234,<p>I think Krapp may be a story-teller rather ...,2019-03-01 06:21:08.014,2019-03-01 06:21:08.014,NaN,1,NaN,177422,Week Eight Discussion Thread,NaN,2019-01-15 17:42:16.549,2019-02-22 15:53:24.503,NaN,f,NaN,NaN,Winter 2019


In [9]:
discussion_post_2018_2020.groupby(by='term')['discussion_post_id'].count().reset_index(name='discussion_post_cnt').reindex([0, 4, 2, 1, 5, 3])

,term,discussion_post_cnt
0,Fall 2018,116429
4,Winter 2019,136596
2,Spring 2019,136121
1,Fall 2019,145023
5,Winter 2020,168888
3,Spring 2020,399355


# Clean the data

In [2]:
import online_discourse_analysis as oda
import numpy as np
import pandas as pd

In [3]:
df_small = pd.read_csv('discussion_post_2020_2022_small.csv')

In [4]:
df_small = oda.discussion_text_clean(df_small)

In [5]:
df_small.head(5)

,Unnamed: 0,discussion_post_id,mellon_id,canvas_course_id,discussion_post_content,discussion_post_created_at,discussion_post_updated_at,discussion_post_deleted_at,discussion_post_depth,parent_discussion_post_id,...,discussion_topic_title,is_announcement,discussion_topic_created_at,discussion_topic_posted_at,discussion_topic_delayed_post_at,discussion_topic_pinned,user_group_id,assignment_id,term,cleaned_discussion_post
0,747902,3197026,174257,30808,"<p>Hi Shikha, </p>\n<p><span style=""font-weigh...",2020-11-16 00:38:32.972,2020-11-16 00:38:32.972,NaN,2,3164718.0,...,Week 6 Discussion: The Presidency - Daniel Fir...,NaN,2020-09-29 04:07:28.75,2020-09-29 04:07:28.75,NaN,f,97982.0,590974.0,Fall 2020,"Hi Shikha, I was thinking the same thing--Trum..."
1,388954,3262754,198980,30233,<p>I agree that the cinematography in Amélie i...,2020-11-26 03:11:47.474,2020-11-26 03:11:47.474,NaN,2,3262634.0,...,Amélie - Discussion,NaN,2020-11-23 17:21:21.975,NaN,NaN,f,NaN,629176.0,Fall 2020,I agree that the cinematography in Amélie is a...
2,1040343,5447579,174702,44877,"<p>Hi Neel,</p>\n<p>Thanks for sharing. I real...",2022-05-28 05:48:48.401,2022-05-28 05:49:44.134,NaN,2,5445502.0,...,Discussion Activity 9 - Writing an introduction,NaN,2022-03-28 17:56:06.609,2022-03-28 17:56:06.611,NaN,t,NaN,954867.0,Spring 2022,"Hi Neel,Thanks for sharing. I really like the ..."
3,582815,3974906,163218,36177,<p><span>The cost of installing solar photovol...,2021-04-17 00:56:55.632,2021-04-17 01:05:53.812,NaN,1,NaN,...,The Learning Curve (Graded) - Discussion Forum 6,NaN,2021-04-04 18:29:59.639,2021-04-04 18:29:59.642,NaN,f,123229.0,743314.0,Spring 2021,The cost of installing solar photovoltaic syst...
4,857025,3941691,177766,36262,<p>In the article “Murder in America: A Histor...,2021-04-12 06:30:26.786,2021-04-12 06:30:26.786,NaN,1,NaN,...,Week 2 - Small Group Discussion - Small Learni...,NaN,2021-03-25 16:45:14.966,2021-03-25 16:45:14.969,NaN,f,123212.0,726781.0,Spring 2021,In the article “Murder in America: A Historian...


# Create term/quarter column

In [14]:
df = oda.create_term_col(df_small)

In [6]:
len(df[df['term'] == '0'])

690

In [10]:
df.head(5)

,Unnamed: 0,Unnamed: 0.1,discussion_post_id,mellon_id,canvas_course_id,discussion_post_content,discussion_post_created_at,discussion_post_updated_at,discussion_post_deleted_at,discussion_post_depth,...,discussion_topic_id,discussion_topic_title,is_announcement,discussion_topic_created_at,discussion_topic_posted_at,discussion_topic_delayed_post_at,discussion_topic_pinned,user_group_id,assignment_id,term
0,818696,818696,3710696,184024,33621,<p>Going through the Kriger et. al reading sol...,2021-02-22 02:21:08.093000+00:00,2021-02-22 02:21:08.093,NaN,1,...,465175,Week 7 RPR,NaN,2020-12-28 17:13:43.494,2021-02-15 04:25:16.831000+00:00,NaN,f,NaN,654138.0,Winter 2021
1,1163402,1163402,3280790,189732,29719,<p>The drug I am looking for is loratadine. Th...,2020-11-30 06:14:56.718000+00:00,2020-11-30 06:14:56.718,NaN,1,...,408802,Week 8 Discussion 1 (Side-effects) - Group 27,NaN,2020-10-01 01:43:16.391,2020-10-01 01:43:16.391000+00:00,NaN,f,98950.0,575833.0,Fall 2020
2,1069337,1069337,3937321,180683,35411,<p>I totally agree with the idea that stress w...,2021-04-12 00:23:58.330000+00:00,2021-04-12 00:23:58.33,NaN,1,...,521980,"Discussion 2: The Etiology, Physiology, Sympto...",NaN,2021-03-20 00:55:55.57,2021-03-20 00:55:55.570000+00:00,NaN,f,122281.0,709312.0,Spring 2021
3,279733,279733,3419760,173435,34214,<p>Hello everyone!! My name is Darshit Shethna...,2021-01-09 10:59:23.040000+00:00,2021-01-09 10:59:54.513,NaN,1,...,478022,INTRODUCTIONS,NaN,2021-01-06 07:09:05.054,2021-01-06 07:09:05.469000+00:00,NaN,f,NaN,675548.0,Winter 2021
4,1055184,1055184,3812426,196511,34162,<p><span>Hypoactive Sexual Desire Disorder (AK...,2021-03-11 06:52:05.860000+00:00,2021-03-11 06:52:05.86,NaN,1,...,512474,Discussion 10: Health & Medicine - Group 4,NaN,2021-03-08 03:16:04.695,2021-03-08 03:16:04.698000+00:00,NaN,f,110753.0,702805.0,Winter 2021


# Calculate Behavioral Features

In [15]:
df = oda.discussion_topic_post_time_gap(df)

In [17]:
df = oda.calculate_features_avg(df, ['discussion_topic_post_time_gap'])

In [8]:
df = oda.discussion_post_count(df)

In [18]:
df.head(10)

,term,discussion_topic_post_time_gap_avg
0,0,30.423914
1,Fall 2020,21.495316
2,Fall 2021,27.240580
3,Spring 2021,19.848608
4,Spring 2022,27.877707
5,Summer 2021,17.694850
6,Winter 2021,17.389650
7,Winter 2022,19.990592


# Calculate Reply Rates

In [9]:
df = pd.read_csv('discussion_post_2020_2022_small.csv')

In [10]:
df = oda.discussion_reply_rate(df)

In [11]:
df1 = df[['discussion_post_id', 'discussion_post_content', 'reply_rate']]

In [12]:
df1 = oda.discussion_text_clean(df1)

/root/Discussion_Behavior_Project/online_discourse_analysis.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['cleaned_discussion_post'] = df['discussion_post_content'].str.replace(regex, '')


# LDA Models

## High Reply Rate Analysis

In [6]:
!pip3 install -U nltk
!pip3 install stop-words
!pip3 install gensim

  Using cached stop_words-2018.7.23-py3-none-any.whl
  Using cached gensim-4.2.0-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (24.1 MB)
  Using cached smart_open-6.0.0-py3-none-any.whl (58 kB)


In [75]:
from nltk.tokenize import RegexpTokenizer
from stop_words import get_stop_words
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim

In [76]:
tokenizer = RegexpTokenizer(r"[\w']+")

# create English stop words list
en_stop = get_stop_words('en')

# Create p_stemmer of class PorterStemmer
p_stemmer = PorterStemmer()

In [49]:
df_hr = df1[df1['reply_rate'] > 2]

In [50]:
df_hr.shape

(23, 4)

In [51]:
df_hr.head(5)

,discussion_post_id,discussion_post_content,reply_rate,cleaned_discussion_post
81,3545388,<p><span>PROMPT 4 You are invited to initiate...,5,PROMPT 4 You are invited to initiate your own...
162,3993887,<p>Maslow's hierarchy generally fits with my e...,3,Maslow's hierarchy generally fits with my expe...
276,3863600,"<p><iframe style=""width: 400px; height: 225px;...",4,;;
372,4232164,"<p>Interestingly enough, social disputes can b...",3,"Interestingly enough, social disputes can be s..."
578,4041433,"<p>When I think about social media platforms, ...",3,"When I think about social media platforms, the..."


In [16]:
for i in range(0, 5):
    print(df_hr['cleaned_discussion_post'].iloc[i])
    print()

PROMPT 4  You are invited to initiate your own discussion by selecting a portion of an assigned text to comment on that wasn't addressed in the prompts above.  This discussion may include asking questions about what an author meant, disputing their conclusions, or explaining what their ideas mean to you on a personal level. Culture War/James Hunter/Page 225In his essay, James Hunter says that "the mass media act act as a filter through which our perceptions of the world around us take shape" (Hunter 225). He believes that the mass media has the power to change and reshape people' mind, perception and spiritual value. And as he claims in the following essay, those who own the mass media would also hold enormous power at the same time. In many cases, the groups hold the power of mass media are more conservative because they want to keep their value on the main stage instead of being taken place by something new. In that case, when there is a new culture value emerging, the conservatives 

In [52]:
# compile the cleaned_discussion_post column into a list
doc_set = [str(x) for x in list(df_hr['cleaned_discussion_post'].values)]
print(len(doc_set))

23


In [77]:
# list for tokenized documents in loop
texts = []

In [78]:
# loop through document list
for content in doc_set:
    
    # clean and tokenize document string
    raw = content.lower()
    tokens = tokens = tokenizer.tokenize(raw)

    # remove stop words from tokens
    stopped_tokens = [w for w in tokens if w not in en_stop]
    
    # stem tokens
    stemmed_tokens = [p_stemmer.stem(i) for i in stopped_tokens]
    
    # add tokens to list
    texts.append(stemmed_tokens)

In [79]:
# turn our tokenized documents into a id <-> term dictionary
dictionary = corpora.Dictionary(texts)
    
# convert tokenized documents into a document-term matrix
corpus = [dictionary.doc2bow(text) for text in texts]

In [80]:
# generate LDA model
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics=5, id2word = dictionary, passes=1)

In [81]:
print(ldamodel.print_topics(num_topics=5, num_words=4))

[(0, '0.007*"movi" + 0.006*"class" + 0.005*"use" + 0.005*"year"'), (1, '0.008*"read" + 0.008*"time" + 0.006*"just" + 0.006*"women"'), (2, '0.011*"mass" + 0.010*"group" + 0.009*"power" + 0.009*"media"'), (3, '0.012*"group" + 0.010*"make" + 0.007*"person" + 0.006*"use"'), (4, '0.011*"senior" + 0.010*"http" + 0.009*"stress" + 0.009*"retriev"')]


In [82]:
for i in range(0, 4):
    print(texts[i])
    print()

['prompt', '4', 'invit', 'initi', 'discuss', 'select', 'portion', 'assign', 'text', 'comment', 'address', 'prompt', 'discuss', 'may', 'includ', 'ask', 'question', 'author', 'meant', 'disput', 'conclus', 'explain', 'idea', 'mean', 'person', 'level', 'cultur', 'war', 'jame', 'hunter', 'page', '225in', 'essay', 'jame', 'hunter', 'say', 'mass', 'media', 'act', 'act', 'filter', 'percept', 'world', 'around', 'us', 'take', 'shape', 'hunter', '225', 'believ', 'mass', 'media', 'power', 'chang', 'reshap', "people'", 'mind', 'percept', 'spiritu', 'valu', 'claim', 'follow', 'essay', 'mass', 'media', 'also', 'hold', 'enorm', 'power', 'time', 'mani', 'case', 'group', 'hold', 'power', 'mass', 'media', 'conserv', 'want', 'keep', 'valu', 'main', 'stage', 'instead', 'taken', 'place', 'someth', 'new', 'case', 'new', 'cultur', 'valu', 'emerg', 'conserv', 'alway', 'wield', 'mass', 'media', 'attack', 'new', 'cultur', 'valu', 'order', 'secur', 'will', 'caus', 'cultur', 'conflict', 'upgrad', 'cultur', 'war', 

# Remove running query

In [ ]:
redshift_s3_connect.view_running_sql_commands()

In [ ]:
pid = 'pid'
redshift_s3_connect.cancel_sql_command(pid)